# Installations/Préparations

## install

In [1]:
!pip install importlib-metadata==4.13.0
#!pip install setuptools==65.5.0
#!pip install git+https://github.com/osigaud/bbrl_gym
!pip install git+https://github.com/osigaud/bbrl
!pip install omegaconf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/osigaud/bbrl to /tmp/pip-req-build-szkkqwpy
  Running command git clone --filter=blob:none --quiet https://github.com/osigaud/bbrl /tmp/pip-req-build-szkkqwpy
  Resolved https://github.com/osigaud/bbrl to commit cb2c22b82bfedfb04d8232ba432cdbd798fd797a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## import

In [2]:
from bbrl.workspace import Workspace
from bbrl import get_class, get_arguments, instantiate_class

#import bbrl_gym
import gym

from bbrl.agents.agent import Agent
from bbrl.agents import Agents, TemporalAgent, PrintAgent
from bbrl.agents.gymb import NoAutoResetGymAgent

from bbrl.utils.replay_buffer import ReplayBuffer
from bbrl.utils.chrono import Chrono

from bbrl.visu.visu_policies import plot_policy
from bbrl.visu.visu_critics import plot_critic
from bbrl.visu.common import final_show

from omegaconf import OmegaConf
from omegaconf import DictConfig

import torch
import torch.nn as nn
import torch.nn.functional as F

import random
import numpy as np
import time
import matplotlib
import os
import functools
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import copy

# Implémentations depuis bbrl_examples

In [3]:
class Logger:
    def __init__(self, cfg):
        self.logger = instantiate_class(cfg.logger)

    def add_log(self, log_string, loss, epoch):
        self.logger.add_scalar(log_string, loss.item(), epoch)

    # Log losses
    def log_losses(self, epoch, critic_loss, entropy_loss, actor_loss):
        self.add_log("critic_loss", critic_loss, epoch)
        self.add_log("entropy_loss", entropy_loss, epoch)
        self.add_log("actor_loss", actor_loss, epoch)

    def log_reward_losses(self, rewards, nb_steps):
        self.add_log("reward/mean", rewards.mean(), nb_steps)
        self.add_log("reward/max", rewards.max(), nb_steps)
        self.add_log("reward/min", rewards.min(), nb_steps)
        self.add_log("reward/median", rewards.median(), nb_steps)

def format_num(num, pos):
    # Pos is a required parameter, but it is not used
    magnitude = 0
    labels = ["", "K", "M", "G"]
    while abs(num) >= 1e3:
        magnitude += 1
        num /= 1e3

    return f"{num:.1f}{labels[magnitude]}"

class Plotter:
    def __init__(self, steps_filename, rewards_filename):
        self.steps_filename = steps_filename
        self.rewards_filename = rewards_filename

    def plot_reward(
        self,
        algo_name,
        env_name,
        mode="mean",
        prefix="",
        suffix=".pdf",
        save_fig=True,
        save_dir="./plots/",
    ):
        _, ax = plt.subplots(figsize=(9, 6))
        formatter = FuncFormatter(format_num)

        colors = ["#09b542", "#008fd5", "#fc4f30", "#e5ae38", "#e5ae38", "#810f7c"]
        color = colors[0]

        loader = RewardLoader(self.steps_filename, self.rewards_filename)
        steps, rewards = loader.load()
        print(steps, rewards)
        # steps, rewards = equalize_lengths(steps, rewards)

        if mode == "best":
            best = rewards.sum(axis=1).argmax()
            mean = rewards[best]
        elif mode == "max":
            mean = np.max(rewards, axis=0)
        else:
            std = rewards.std(axis=0)
            mean = rewards.mean(axis=0)
            ax.fill_between(steps, mean + std, mean - std, alpha=0.1, color=color)
        ax.plot(steps, mean, lw=2, label=f"{algo_name}", color=color)
        ax.xaxis.set_major_formatter(formatter)
        plt.legend()

        save_dir += f"{env_name}/"

        clean_env_name = env_name.split("-")[0]
        figure_name = f"{prefix}{clean_env_name.lower()}_{mode}"
        title = f"{clean_env_name} ({mode})"
        if suffix:
            figure_name += f"{suffix}"
        final_show(save_fig, True, save_dir, figure_name, "timesteps", "rewards", title)

    def plot_histograms(
        self,
        rewards,
        env_name,
        suffix="",
        save_dir="./plots/",
        plot=True,
        save_fig=True,
    ):
        plt.figure(figsize=(9, 6))

        colors = ["#09b542", "#008fd5", "#fc4f30", "#e5ae38", "#e5ae38", "#810f7c"]
        # colors = ["#fc4f30", "#008fd5", "#e5ae38"]

        n_bars = len(rewards)
        x = np.arange(len(list(rewards.values())[0]))
        width = 0.75 / n_bars

        for i, reward in enumerate(rewards.values()):
            plt.bar(x + width * i, np.sort(reward)[::-1], width=width, color=colors[i])

        plt.legend(labels=rewards.keys())
        plt.xticks([], [])

        save_dir += f"{env_name}/"

        clean_env_name = env_name.split("-")[0]
        title = clean_env_name
        figure_name = f"{clean_env_name.lower()}-histograms"

        if suffix:
            title += f" ({suffix})"
            figure_name += f"{suffix}"

        final_show(save_fig, plot, save_dir, figure_name, "", "rewards", title)

class RewardLogger:
    def __init__(self, steps_filename, rewards_filename):
        self.steps_filename = steps_filename
        self.rewards_filename = rewards_filename
        self.episode = 0
        self.all_rewards = []
        self.all_rewards.append([])
        self.all_steps = []

    def add(self, nb_steps, reward):
        if self.episode == 0:
            self.all_steps.append(nb_steps)
        self.all_rewards[self.episode].append(reward.item())

    def new_episode(self):
        self.episode += 1
        self.all_rewards.append([])

    def save(self):
        # print("reward loader save:", self.all_steps,  self.all_rewards)
        with open(self.steps_filename, "ab") as f:
            np.save(f, self.all_steps)
        with open(self.rewards_filename, "ab") as f:
            np.save(f, self.all_rewards)

class RewardLoader:
    def __init__(self, steps_filename, rewards_filename):
        self.steps_filename = steps_filename
        self.rewards_filename = rewards_filename

    def load(self):
        with open(self.steps_filename, "rb") as f:
            steps = np.load(f, allow_pickle=True)
        with open(self.rewards_filename, "rb") as f:
            rewards = np.load(f, allow_pickle=True)
        return steps, rewards

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

def build_alt_mlp(sizes, activation):
    layers = []
    for j in range(len(sizes) - 1):
        if j < len(sizes) - 2:
            layers += [nn.Linear(sizes[j], sizes[j + 1]), activation]
        else:
            layers += [nn.Linear(sizes[j], sizes[j + 1])]
    return nn.Sequential(*layers)


class DiscreteQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.is_q_function = True
        self.model = build_alt_mlp(
            [state_dim] + list(hidden_layers) + [action_dim], activation=nn.ReLU()
        )

    def forward(self, t, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        q_values = self.model(obs).squeeze(-1)
        self.set(("q_values", t), q_values)
        if choose_action:
            action = q_values.argmax(-1)
            self.set(("action", t), action)

    def predict_action(self, obs, stochastic):
        q_values = self.model(obs).squeeze(-1)
        if stochastic:
            probs = torch.softmax(q_values, dim=-1)
            action = torch.distributions.Categorical(probs).sample()
        else:
            action = q_values.argmax(-1)
        return action

    def predict_value(self, obs, action):
        q_values = self.model(obs).squeeze(-1)
        return q_values[action[0].int()]


class EGreedyActionSelector(Agent):
    def __init__(self, epsilon):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, t, **kwargs):
        q_values = self.get(("q_values", t))
        nb_actions = q_values.size()[1]
        size = q_values.size()[0]
        is_random = torch.rand(size).lt(self.epsilon).float()
        random_action = torch.randint(low=0, high=nb_actions, size=(size,))
        max_action = q_values.max(1)[1]
        action = is_random * random_action + (1 - is_random) * max_action
        action = action.long()
        self.set(("action", t), action)

# Nouveaux Agents & Wrapper

In [5]:
# Vraiment utile ? Wrapper/env inatégnables dans le run
# Non utilisée pour l'instant
class CartWrapper(gym.Wrapper):
    def __init__(self, env):
        super(CartWrapper, self).__init__(env)

        # choix arbitraire d'un goal de départ
        pos = random.uniform(-2.4, 2.4)
        vel = random.uniform(-1e3, 1e3)
        angle = random.uniform(-.2095, .2095)
        angle_vel = random.uniform(-1e3, 1e3)
        # set du paramètre goal de l'environnement
        self.goal = (pos, vel, angle, angle_vel)

        # distance acceptable autour du but
        self.eps = 0.5

    def step(self, action):
        # effectue un step de l'env CartPole
        next_state, _, _, info = self.env.step(action)

        # calcul de distance entre next_state et le but
        distance = np.linalg.norm(next_state - self.goal)

        # récompense et fin si distance inférieure à eps
        if distance < self.eps:
            reward = 1
            done = True
        else:
            reward = 0
            done = False

        return next_state, reward, done, info


class GoalRelabellingAgent(Agent):
    def __init__(self):
        super().__init__()

    def forward(self, t, desired_goal=None, achieved_goal=None, choice=False, **kwargs):
        if desired_goal != None:
            # modification du but désiré
            self.set(("env/desired_goal", t), desired_goal)

        if achieved_goal != None:
            # modification du but atteind
            self.set(("env/achieved_goal", t), achieved_goal)

        if choice:
            # choix arbitraire d'un goal
            pos = random.uniform(-2.4, 2.4)
            vel = random.uniform(-1e3, 1e3)
            angle = random.uniform(-.2095, .2095)
            angle_vel = random.uniform(-1e3, 1e3)

            # set du paramètre goal de l'environnement
            goal = np.array([pos, vel, angle, angle_vel])
            ### TypeError: 'int' object is not callable ?
            self.set(("env/desired_goal", t), goal)


class RewardAgent(Agent):
    def __init__(self):
        super().__init__()

    def forward(self, t, **kwargs):
        if t != 0:
            # récupération des buts atteints et désirés
            desired_goal = self.get(("env/desired_goal", t))
            obs = self.get(("env/env_obs", t))
            eps = [0.5]*len(obs)

            # calcul de distance entre next_state et le but
            distance = np.linalg.norm(desired_goal - obs, axis=1)

            # récompense et fin si distance inférieure à eps
            reward = np.where(distance < eps, 1., 0.)
            done = (distance < eps)

            # set de la reward et du done
            ### TypeError: 'int' object is not callable ?
            self.set(("env/reward", t), reward)
            self.set(("env/done", t), done)


# En cours
class HERAgentFinal(Agent):
    def __init__(self):
        super().__init__()

    def forward(self, t, rb, **kwargs):
        pass


class GCDQNAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.is_q_function = True
        self.model = build_alt_mlp(
            [state_dim +4] + list(hidden_layers) + [action_dim], activation=nn.ReLU()   # +4 car taille d'un état ?
        )

    def forward(self, t, choose_action=True, **kwargs):
        # récupération des valeurs pour input
        obs = self.get(("env/env_obs", t))
        goal = self.get(("env/desired_goal", t))
        agent_input = torch.cat([obs, goal], dim=1)

        # calcul des valeurs
        q_values = self.model(agent_input).squeeze(-1)
        self.set(("q_values", t), q_values)

        if choose_action:
            action = q_values.argmax(-1)
            self.set(("action", t), action)

    def predict_action(self, obs, stochastic):
        q_values = self.model(obs).squeeze(-1)
        if stochastic:
            probs = torch.softmax(q_values, dim=-1)
            action = torch.distributions.Categorical(probs).sample()
        else:
            action = q_values.argmax(-1)
        return action

    def predict_value(self, obs, action):
        q_values = self.model(obs).squeeze(-1)
        return q_values[action[0].int()]

# Set-Up

## Paramètres

In [6]:
params={
  "save_best": False,
  "plot_agents": True,
  
  "logger":{
    "classname": "bbrl.utils.logger.TFLogger",
    "log_dir": "./dqn_logs/",
    "cache_size": 10000,
    "every_n_seconds": 1,
    "verbose": False,    
  },

  "algorithm":{
    "seed": 3,                      # modifié par la main loop
    "nb_seeds": 1,                  # nb de seed testées (de 0 à valeur proposée)

    "epsilon": 0.05,                 # valeur pour epsilon-greedy
    "discount_factor": 0.99,        # delta
    "gae": 0.8,                     # ???

    "n_steps": 64,                  # nb max de step par épisode ?
    "n_envs": 10,                    # nb d'environnement en simultané
    "n_episodes": 20,                # nb d'épisodes
    "nb_evals": 5,                 # nb d'évaluation après train
    "eval_interval": 10,            # intervalle (steps) entre évaluations ?
    "target_critic_update": 10,    # intervalle (steps) entre chaque maj de Q_target ?

    "learning_starts": 1,           # ???

    "n_updates": 20,                # nb d'update par le Replay Buffer
    "buffer_size": 1e6,             # taille max du Replay Buffer
    "batch_size": 50,              # taille du batch Replay Buffer

    "max_grad_norm": 0.5,           # ???
    "architecture":{"hidden_size": [128, 128]},
  },

  "gym_env":{
    "classname": "__main__.make_gym_env",
    "env_name": "CartPole-v1"
  },

  "optimizer":{
    "classname": "torch.optim.Adam",
    "lr": 2.3e-3,
  }
}

config = OmegaConf.create(params)

## Utils

In [7]:
def make_gym_env(env_name):
    #env = CartWrapper(gym.make(env_name))
    #return env
    return gym.make(env_name)


def get_env_agents(cfg):
    train_env_agent = NoAutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.n_envs,
        cfg.algorithm.seed,
    )
    # print_agent = PrintAgent()
    eval_env_agent = NoAutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.nb_evals,
        cfg.algorithm.seed,
    )
    return train_env_agent, eval_env_agent


def create_dqn_agent(cfg, train_env_agent, eval_env_agent):
    obs_size, act_size = train_env_agent.get_obs_and_actions_sizes()

    critic = GCDQNAgent(obs_size, cfg.algorithm.architecture.hidden_size, act_size)
    explorer = EGreedyActionSelector(cfg.algorithm.epsilon)
    target_critic = copy.deepcopy(critic)

    goal_label_agent = TemporalAgent(GoalRelabellingAgent())
    reward_agent = RewardAgent()
    her_agent = TemporalAgent(HERAgentFinal())

    q_agent = TemporalAgent(critic)
    target_q_agent = TemporalAgent(target_critic)
    
    tr_agent = Agents(train_env_agent, critic, explorer, reward_agent)
    ev_agent = Agents(eval_env_agent, critic, reward_agent)

    # Get an agent that is executed on a complete workspace
    train_agent = TemporalAgent(tr_agent)
    eval_agent = TemporalAgent(ev_agent)

    train_agent.seed(cfg.algorithm.seed)
    return train_agent, eval_agent, q_agent, target_q_agent, goal_label_agent, her_agent


# Configure the optimizer
def setup_optimizers(cfg, q_agent):
    optimizer_args = get_arguments(cfg.optimizer)
    parameters = q_agent.parameters()
    optimizer = get_class(cfg.optimizer)(parameters, **optimizer_args)
    return optimizer


def compute_critic_loss(cfg, reward, must_bootstrap, q_values, target_q_values, action):
    # Compute temporal difference
    max_q = target_q_values[1].max(-1)[0].detach()

    target = (
        reward[:-1]
        + cfg.algorithm.discount_factor * max_q * must_bootstrap.int()
    )

    vals = q_values.squeeze()
    qvals = torch.gather(vals, dim=1, index=action)
    qvals = qvals[:-1]

    mse = nn.MSELoss()
    critic_loss = mse(target, qvals)
    return critic_loss

# Méthode Run DQN

In [8]:
def run_dqn(cfg, reward_logger):
    # 1)  Build the  logger
    logger = Logger(cfg)
    best_reward = -10e9

    # 2) Create the environment agent
    train_env_agent, eval_env_agent = get_env_agents(cfg)

    # 3) Create the DQN-like Agent
    #train_agent, eval_agent, q_agent, target_q_agent = create_dqn_agent(cfg, train_env_agent, eval_env_agent)
    train_agent, eval_agent, q_agent, target_q_agent, goal_label_agent, her_agent = create_dqn_agent(cfg, train_env_agent, eval_env_agent)

    # 4) Create the Replay Buffer Agent
    rb = ReplayBuffer(max_size=cfg.algorithm.buffer_size)

    # 6) Configure the optimizer
    optimizer = setup_optimizers(cfg, q_agent)
    nb_steps = 0
    tmp_steps = 0
    tmp_steps2 = 0

    # 7) Training
    # Train des épisodes
    for _ in range(cfg.algorithm.n_episodes):
        train_workspace = Workspace()

        # tirage d'un but pour l'épisode
        goal_label_agent(train_workspace, t=0, choice=True, n_steps=2)
        # train sur le workspace
        train_agent(train_workspace, t=0, stop_variable="env/done", stochastic=True)

        transition_workspace = train_workspace.get_transitions()

        # comptage du nb de step de l'épisode
        action = transition_workspace["action"]
        nb_steps += action[0].shape[0]

        # ajout des transitions au RB
        rb.put(transition_workspace)

        # 7.1) Loop Replay Buffer
        for _ in range(cfg.algorithm.n_updates):
            #her_agent(transition_workspace, t=0, rb=rb, n_steps=2)

            # tirage aléatoire d'un minibatch dans un Workspace
            rb_workspace = rb.get_shuffled(cfg.algorithm.batch_size)

            # modifie le but atteint
            achieved_goal = rb_workspace["env/env_obs"][-1]
            goal_label_agent(rb_workspace, t=0, achieved_goal=achieved_goal, n_steps=2)
            #goal_label_agent(rb_workspace, t=0, desired_goal=achieved_goal, n_steps=2)
            #reward_agent(rb_workspace, t=0, n_steps=2)

            # calcul des Q-values
            # The q agent needs to be executed on the rb_workspace workspace (gradients are removed in workspace).
            q_agent(rb_workspace, t=0, n_steps=2, choose_action=False)

            q_values, done, truncated, reward, action = rb_workspace[
                "q_values", "env/done", "env/truncated", "env/reward", "action"
            ]

            with torch.no_grad():
                target_q_agent(rb_workspace, t=0, n_steps=2, stochastic=True)

            target_q_values = rb_workspace["q_values"]
            # assert torch.equal(q_values, target_q_values), "values differ"

            # Determines whether values of the critic should be propagated
            # True if the episode reached a time limit or if the task was not done
            # See https://colab.research.google.com/drive/1erLbRKvdkdDy0Zn1X_JhC01s1QAt4BBj?usp=sharing
            must_bootstrap = torch.logical_or(~done[1], truncated[1])

            if rb.size() > cfg.algorithm.learning_starts:
                # Compute critic loss
                critic_loss = compute_critic_loss(
                    cfg, reward, must_bootstrap, q_values[0], target_q_values[1], action
                )

                # Store the loss for tensorboard display
                logger.add_log("critic_loss", critic_loss, nb_steps)

                optimizer.zero_grad()
                critic_loss.backward()
                torch.nn.utils.clip_grad_norm_(
                    q_agent.parameters(), cfg.algorithm.max_grad_norm
                )
                optimizer.step()

        # 7.2) Maj du Q_target (sous conditions)
        if nb_steps - tmp_steps2 > cfg.algorithm.target_critic_update:
            tmp_steps2 = nb_steps
            target_q_agent.agent = copy.deepcopy(q_agent.agent)
        
        # 7.3) Évaluation régulère
        if nb_steps - tmp_steps > cfg.algorithm.eval_interval:
            tmp_steps = nb_steps
            eval_workspace = Workspace()  # Used for evaluation
            eval_agent(
                eval_workspace, t=0, stop_variable="env/done", choose_action=True
            )
            rewards = eval_workspace["env/cumulated_reward"][-1]
            mean = rewards.mean()
            logger.add_log("reward", mean, nb_steps)
            print(f"reward: {mean}")
            reward_logger.add(nb_steps, mean)
            
            if cfg.save_best and mean > best_reward:
                best_reward = mean
                directory = "./dqn_critic/"
                
                if not os.path.exists(directory):
                    os.makedirs(directory)
                
                filename = directory + "dqn_" + str(mean.item()) + ".agt"
                eval_agent.save_model(filename)
                
                if cfg.plot_agents:
                    policy = eval_agent.agent.agents[1]
                    plot_policy(
                        policy,
                        eval_env_agent,
                        "./dqn_plots/",
                        cfg.gym_env.env_name,
                        best_reward,
                        stochastic=False,
                    )
                    plot_critic(
                        policy,
                        eval_env_agent,
                        "./dqn_plots/",
                        cfg.gym_env.env_name,
                        best_reward,
                    )

# Main

In [9]:
def main_loop(cfg):
    chrono = Chrono()
    logdir = "./plot/"

    if not os.path.exists(logdir):
        os.makedirs(logdir)

    reward_logger = RewardLogger(
        logdir + "dqn.steps", logdir + "dqn.rwd"
    )

    for seed in range(cfg.algorithm.nb_seeds):
        #cfg.algorithm.seed = seed
        torch.manual_seed(cfg.algorithm.seed)
        run_dqn(cfg, reward_logger)

        if seed < cfg.algorithm.nb_seeds - 1:
            reward_logger.new_episode()

    reward_logger.save()
    chrono.stop()
    plotter = Plotter(logdir + "dqn.steps", logdir + "dqn.rwd")
    plotter.plot_reward("dqn", cfg.gym_env.env_name)

In [10]:
main_loop(config)

/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


TypeError: ignored